This script extracts WMH lesion loads both for global brain and for brain parcellated with the Hammers Atlas. 
Remember that the Hammers Atlas is in MNI space, so before using this script you have to make sure that you ran `transform_HammersAtlas_to_subject_space.ipynb`

In [437]:
import nibabel as nib
import nibabel.imagestats as nibstats

from nilearn import plotting as niplot
from nilearn import datasets
from nilearn.maskers import NiftiLabelsMasker
from nilearn import image as nimg

from bids import BIDSLayout

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as ss

import pickle

In [438]:
BIDS_dir = '/home/riccardo/ADNI_Hopf/Data/ALL'

In [439]:
# set the layout variable to the BIDS-folder where your WMH are located
layout = BIDSLayout(BIDS_dir, validate = False, config = ['bids', 'derivatives'])
subjs = layout.get_subjects()

In [440]:
HC_np = '/home/riccardo/ADNI_Hopf/Results/subject_list_timeseries_HC.npy'
HC_subjs = np.load(HC_np)

MCI_np = '/home/riccardo/ADNI_Hopf/Results/subject_list_timeseries_MCI.npy'
MCI_subjs = np.load(MCI_np)

ALL_np = '/home/riccardo/ADNI_Hopf/Results/subject_list_timeseries_all.npy'
ALL_subjs = np.load(ALL_np)

In [441]:
# iterate through subjects, since not all subjects have WMH masks, perform WMH volume extraction only on those who have the WMH segmentation maskcalled '_label-WMH'

def calculate_WMH_load(subj):
    
    wmh_mask = layout.get(subject = subj, datatype = 'anat', label = 'WMH', return_type='file')
    if wmh_mask != list():
        wmh = nib.load(wmh_mask[0])
        wmh_volume = round(nibstats.mask_volume(wmh),1)
    else:
        wmh_volume = 0
    return wmh_volume

In [442]:
# for subj in subjs:

#     wmh_mask = layout.get(subject = subj, datatype = 'anat', label = 'WMH', return_type='file')
#     if wmh_mask != list():
#         wmh = nib.load(wmh_mask[0])
#         wmh_volume = round(nibstats.mask_volume(wmh),1)
            

In [494]:
out_dir = '/home/riccardo/ADNI_Hopf/Results/'

overall_WMH_dict = {}

for subj in subjs:
    overall_WMH_dict[subj] = calculate_WMH_load(subj)


In [503]:
WMH_subjs_list = []
WMH_burden_list = []

for k, v in overall_WMH_dict.items():
    WMH_subjs_list.append(k)
    WMH_burden_list.append(v)

    WMH_burden_arr = np.array(WMH_burden_list)

normalized_WMH_burden_series = np.round((WMH_burden_arr - np.min(WMH_burden_arr))/np.ptp(WMH_burden_arr), 5)  # Normalize the whole group in [0,1]


overall_WMH_dict_normalized = {k: normalized_WMH_burden_series[n] for n, k in enumerate(WMH_subjs_list)}

# open file for writing, "w" 
f = open("/home/riccardo/ADNI_Hopf/Results/overall_WMH_burden_all_normalized.pkl","wb")
# write json object to file
pickle.dump(overall_WMH_dict_normalized, f)
# close file
f.close()

# open file for writing, "w" 
g = open("/home/riccardo/ADNI_Hopf/Results/overall_WMH_burden_all.pkl","wb")
# write json object to file
pickle.dump(overall_WMH_dict, g)
# close file
g.close()



# WMH_condition_list = []

# for k, v in overall_WMH_dict.items():
#     WMH_subjs_list.append(k[1])
#     WMH_burden_list.append(v)
#     if k[1] in HC_subjs:
#         WMH_condition_list.append('hc')
#     elif k[1] in MCI_subjs:
#         WMH_condition_list.append('mci')
#     else:
#         WMH_condition_list.append('excluded')

# WMH_subjs_series = pd.Series(WMH_subjs_list, dtype = 'string')
# WMH_burden_series = pd.Series(WMH_burden_list, dtype = 'float')
# normalized_WMH_burden_series = (WMH_burden_series - np.min(WMH_burden_series))/np.ptp(WMH_burden_series)  # Normalize the whole group in [0,1]
# WMH_condition_list = pd.Series(WMH_condition_list, dtype = 'string')

# WMH_df = pd.DataFrame([WMH_subjs_series, WMH_burden_series, normalized_WMH_burden_series, WMH_condition_list]).T
# WMH_df.columns = ['Subject', 'Total_WMH_load', 'Normalized_total_WMH_load', 'Condition']


In [444]:
# WMH_df.to_csv('/home/riccardo/ADNI_Hopf/Results/WMH_overall_all.csv', index=False)

# WMH lesion load per parcellation

The problem is that AAL and Desikan are cortical atlases, which makes sense since what we want are interactions between cortical regions, but our WMH loads are mainly subcortical.. So basically if I use AAL parcellations and I try to intersect them with my WMH lesion load I don't find anything because they do not intersect. Trying with the Hammers atlas that has WM as well as GM, could be a start. Downloaded from [here](http://brain-development.org/brain-atlases/adult-brain-atlases/adult-brain-maximum-probability-map-hammers-mith-atlas-n30r83-in-mni-space/#Download_-_Adult_brain_maximum_probability_mapWhite_matter_atlas_in_MNI-space).

In [445]:
BIDS_dir = '/home/riccardo/WMH/ALL/'
out_dir = '/home/riccardo/Documents/Hopf_model_AD/Results/'

In [446]:
# Load the labels file from where you stored it
hammer_labels_file = '/home/riccardo/ADNI_Hopf/Utils/Hammers67n20/n30r83_names_spreadsheet.csv'
#get the labels numbers
hammer_labels_df = pd.read_csv(hammer_labels_file, index_col=None)
hammer_labels = hammer_labels_df.iloc[1:,0].to_dict()

In [447]:
def get_T1(subj):
    '''Get the T1 file for the subject'''
    all_t1_filenames = layout.get(subject = subj, datatype = 'anat', desc = 'preproc', extension = 'nii.gz', return_type='file')
    mni2009_filenames = layout.get(subject = subj, datatype = 'anat', desc = 'preproc', space = 'MNI152NLin2009cAsym', extension = 'nii.gz', return_type='file')
    mni6_filenames = layout.get(subject = subj, datatype = 'anat', desc = 'preproc', space = 'MNI152NLin6Asym', extension = 'nii.gz', return_type='file')
    t1_filenames = [f for f in all_t1_filenames if f not in mni2009_filenames + mni6_filenames if 'FLAIR' not in f]
    t1_filename = t1_filenames[0]
    return t1_filename

#define our function to extract WMH lesion load for each patient for all patients
# def extract_regional_WMH_load(subj, t1_img, labels = hammer_labels):

#     '''This function loops through the subjects in our BIDS folder and checks if there are WMH lesion masks available. If there are, it calculates the WMH lesion load for each
#     parcellation in the HammersAtlas in subject space and returns an array containing the volume per region'''

#     # Get the transformed Hammers atlas in subject space
#     hammers_atlas_filenames = layout.get(subject = subj, datatype = 'anat', desc = 'brain', suffix = 'HammersAtlasParcellation', return_type='file')
#     hammers_atlas_file_MNI = layout.get(subject = subj, datatype = 'anat', space = 'MNI152NLin2009cAsym', desc = 'brain', suffix = 'HammersAtlasParcellation', return_type='file')
#     hammers_atlas_file = [hammers for hammers in hammers_atlas_filenames if hammers not in hammers_atlas_file_MNI]
#     hammers_atlas = hammers_atlas_file[0]
#     # Load the image
#     hammers_img = nib.load(hammers_atlas)
#     # Create an empty 1 X 5 numpy array to store the results of the WMH load for each of the considered regions (frontal, temporal, occipital, parietal, cingulum)
#     subj_WMH = np.zeros([1, 5])
#     # check if they have WMH, if not skip
#     wmh_mask = layout.get(subject = subj, datatype = 'anat', label = 'WMH', return_type='file')
#     if wmh_mask != list():
#         wmh = nib.load(wmh_mask[0])

#     # resample the atlas for this patient.... not sure if needed, need to tryout
#         resamp_atlas = nimg.resample_to_img(hammers_img, t1_img, 'nearest')    
#     # calculate WMH lesion load for each region of interest/parcellation for this patient

#         for n, label in enumerate(labels):
#             roi_array_bool = (resamp_atlas.get_fdata() == label)
#             roi_current_array = roi_array_bool.astype(int)
#             roi_current_mask = nib.Nifti1Image(roi_current_array, affine=hammers_img.affine)
#             intersection_mask = nimg.math_img('a * b', a = roi_current_mask, b = wmh)
#             subj_WMH[n] = np.array([round(nibstats.mask_volume(intersection_mask),1)])
#     else:
#         for m, label in enumerate(labels):
#             subj_WMH[m] = 0
#     #     #niplot.plot_roi(roi_img=intersection_mask, alpha=0.4, title=1)
#     return subj_WMH

In [448]:
subjs = layout.get_subjects()

hammers_temporal_lobe_idx = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 30, 31, 82, 83]
hammers_frontal_lobe_idx = [28, 29, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 68, 69, 70, 71, 72, 73, 76, 77, 78, 79, 80, 81 ]
hammers_occipital_lobe_idx = [64, 65, 66, 67, 22, 23]
hammers_parietal_lobe_idx = [60, 61, 62, 63, 32, 33]
hammers_insula_cingulate_idx = [20, 21, 24, 25, 26, 27]

def extract_regional_WMH_load(subj, t1_img, labels = hammer_labels):

    hammers_atlas_filenames = layout.get(subject = subj, datatype = 'anat', desc = 'brain', suffix = 'HammersAtlasParcellation', return_type='file')
    hammers_atlas_file_MNI = layout.get(subject = subj, datatype = 'anat', space = 'MNI152NLin2009cAsym', desc = 'brain', suffix = 'HammersAtlasParcellation', return_type='file')
    hammers_atlas_file = [hammers for hammers in hammers_atlas_filenames if hammers not in hammers_atlas_file_MNI]
    hammers_atlas = hammers_atlas_file[0]
    # Load the image
    hammers_img = nib.load(hammers_atlas)

    # Create an empty 1 X 5 numpy array to store the results of the WMH load for each of the considered regions (frontal, temporal, occipital, parietal, cingulum)
    subj_WMH = np.zeros([len(hammer_labels)])
    # check if they have WMH, if not skip
    wmh_mask = layout.get(subject = subj, datatype = 'anat', label = 'WMH', return_type='file')
    if wmh_mask != list():
        wmh = nib.load(wmh_mask[0])

    # calculate WMH lesion load for each region of interest/parcellation for this patient

        for n, label in enumerate(hammer_labels):
            roi_array_bool = (hammers_img.get_fdata() == label)
            roi_current_array = roi_array_bool.astype(int)
            roi_current_mask = nib.Nifti1Image(roi_current_array, affine=hammers_img.affine)
            intersection_mask = nimg.math_img('a * b', a = roi_current_mask, b = wmh)
            subj_WMH[n] = np.array([round(nibstats.mask_volume(intersection_mask),1)])
    else:
        for m, label in enumerate(hammer_labels):
            subj_WMH[m] = 0


    WMH_temporal = []
    WMH_frontal = []
    WMH_parietal = []
    WMH_occipital = []
    WMH_insula_cingulate = []

    for i, WMH in enumerate(subj_WMH):
    
        if i in hammers_temporal_lobe_idx:
            WMH_temporal.append(WMH)
        elif i in hammers_frontal_lobe_idx:
            WMH_frontal.append(WMH)
        elif i in hammers_parietal_lobe_idx:
            WMH_parietal.append(WMH)
        elif i in hammers_occipital_lobe_idx:
            WMH_occipital.append(WMH)
        elif i in hammers_insula_cingulate_idx:
            WMH_insula_cingulate.append(WMH)

    WMH_temporal_array = np.array(WMH_temporal)
    WMH_frontal_array = np.array(WMH_frontal)
    WMH_parietal_array = np.array(WMH_parietal)
    WMH_occipital_array = np.array(WMH_occipital)
    WMH_insula_cingulate_array = np.array(WMH_insula_cingulate)

    lobe_WMH = {'temporal': np.round(WMH_temporal_array.mean(), 3), 
                'frontal': np.round(WMH_frontal_array.mean(), 3), 
                'parietal': np.round(WMH_parietal_array.mean(), 3),
                'occipital': np.round(WMH_occipital_array.mean(), 3),
                'insula-cingulate': np.round(WMH_insula_cingulate_array.mean(), 3)}
                
    return lobe_WMH

In [449]:
subjs = layout.get_subjects()

subjects_WMH = {}

for i, subj in enumerate(subjs):
    print(f'Processing subject: sub-{subj} ({i+1}/{len(subjs)})')
    t1_img = get_T1(subj)
    subjects_WMH[subj] = extract_regional_WMH_load(subj, t1_img)  
print('Done!') 

Processing subject: sub-ADNI011S6367 (1/33)


/tmp/ipykernel_14522/1631462985.py:30: FutureWarning: Image data has type int64, which may cause incompatibilities with other tools. This will error in NiBabel 5.0. This warning can be silenced by passing the dtype argument to Nifti1Image().
  roi_current_mask = nib.Nifti1Image(roi_current_array, affine=hammers_img.affine)
/home/riccardo/anaconda3/envs/neurolib/lib/python3.9/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)


Processing subject: sub-ADNI037S6083 (2/33)
Processing subject: sub-ADNI068S2187 (3/33)
Processing subject: sub-ADNI002S4799 (4/33)
Processing subject: sub-ADNI002S4654 (5/33)
Processing subject: sub-ADNI002S5178 (6/33)
Processing subject: sub-ADNI002S1155 (7/33)
Processing subject: sub-ADNI037S4706 (8/33)
Processing subject: sub-ADNI003S6014 (9/33)
Processing subject: sub-ADNI003S6432 (10/33)
Processing subject: sub-ADNI003S6606 (11/33)
Processing subject: sub-ADNI037S4214 (12/33)
Processing subject: sub-ADNI011S6618 (13/33)
Processing subject: sub-ADNI003S6258 (14/33)
Processing subject: sub-ADNI024S6385 (15/33)
Processing subject: sub-ADNI024S4674 (16/33)
Processing subject: sub-ADNI006S6651 (17/33)
Processing subject: sub-ADNI100S4556 (18/33)
Processing subject: sub-ADNI003S6259 (19/33)
Processing subject: sub-ADNI019S6186 (20/33)
Processing subject: sub-ADNI024S6033 (21/33)
Processing subject: sub-ADNI003S6307 (22/33)
Processing subject: sub-ADNI003S6268 (23/33)
Processing subject

In [450]:
import csv
# open the csv containing the names of the 78 cortical regions that we will use 
with open('/home/riccardo/ADNI_Hopf/Utils/aal_regions_included.csv', newline='') as f:
    reader = csv.reader(f)
    aal_included_regions = list(reader)
f.close()
#create a 1-D array with the names so that it is more easily iterable
aal_included_regions_array = np.array([aal_included_regions]).flatten()

# Get the AAL atlas labels
import nilearn.datasets as datasets
aal = datasets.fetch_atlas_aal()
labels = np.array(aal.labels)
# create an array with the indices of each label (note that these are not the label number from the nifti image)
indices = np.array([i for i in enumerate(labels)])
SC_regions_index = np.isin(labels, aal_included_regions)
# filter the indices that we want based on the position so to have a final SC matrix only for the regions we considered.
SC_78_regions_aal_atlas = indices[SC_regions_index]
filter_SC = np.array([int(i) for i in SC_78_regions_aal_atlas[:,0]])

aal_frontal_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
aal_insula_cingulate_indices =[28, 29, 30, 31, 32, 33, 34, 35]
aal_temporal_indices = [36, 37, 54, 55, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]
aal_occipital_indices = [42, 43, 44, 45, 46, 47, 48, 49, 50 ,51, 52, 53]
aal_parietal_indices = [56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69]

In [451]:
aal_regional_indices = [aal_frontal_indices, aal_insula_cingulate_indices, aal_temporal_indices, aal_occipital_indices, aal_parietal_indices]
aal_region_name = ['frontal', 'insula-cingulate', 'temporal', 'occipital', 'parietal']

def get_lobe_WMH(aal_regional_indices, aal_region_name):

    """This function is used to convert the regional weights from each region of the Hammers atlas (that also has deep structures) into a 1 X 5 array containing the same weight
    for each region that forms a lobe (e.g., all regions composing the frontal lobe get the same WMH weight)
    
    Inputs:
    aal_regional_indices (list): a list of lists containing the indices that make up each lobe (e.g., [1,2,3,4,...] for the frontal lobe)
    aal_region_name (list): a list of strings with the corresponding lobe names (e.g., ['frontal', 'parietal', etc.])

    Returns:
    WMH_all_arr (nd.array): a 1 X 78 array. For each element of the AAL atlas belonging to a lobe, the output is the average WMH load for that brain lobe.
     """

    WMH_all = []

    for n, v in enumerate(aal_regional_indices):
        WMH_region_aal = np.ones(len(v))
        for i, WMH in np.ndenumerate(WMH_region_aal):
            WMH_region_aal[i] = WMH_region_aal[i] * subjects_WMH[subj][aal_region_name[n]]
            WMH_all.append(WMH_region_aal[i])
    WMH_all_arr = np.array(WMH_all)

    return WMH_all_arr

In [466]:
subjects_WMH_lobewise_dict = {}

for subj in subjs:
    subjects_WMH_lobewise_dict[subj] = get_lobe_WMH(aal_regional_indices, aal_region_name)


In [468]:
wmBurden = np.array([])
for k, v in subjects_WMH_lobewise_dict.items():
    wmBurden = np.concatenate([wmBurden, v]) 

wmBurdenNorm = (wmBurden - np.min(wmBurden))/np.ptp(wmBurden)  # Normalize the whole group in [0,1]
wmBurdenNorm_reshaped = wmBurdenNorm.reshape([len(subjects_WMH_lobewise_dict),78])

In [475]:
lobe_WMH_normalized_dict = {}

for k, a in enumerate(subjects_WMH_lobewise_dict.items()):
    lobe_WMH_normalized_dict[a[0]] = np.round(wmBurdenNorm_reshaped[k], 5)

WMH_lobe_subjs = pd.Series(lobe_WMH_normalized_dict.keys(), dtype = 'string')
lobe_WMH_load = pd.Series(lobe_WMH_normalized_dict.values(), dtype = 'object')

In [476]:
# Trying to convert everything to a dataframe, but when I load it I lose the numpy array inside the df even though I pickle-save them.
# WMH_lobe_condition_list = []

# for k, v in lobe_WMH_normalized_dict.items():

#     if k in HC_subjs:
#         WMH_lobe_condition_list.append('hc')
#     elif k in MCI_subjs:
#         WMH_lobe_condition_list.append('mci')
#     else:
#         WMH_lobe_condition_list.append('excluded')

# WMH_lobe_condition_series = pd.Series(WMH_lobe_condition_list, dtype = 'string')
# WMH_lobe_weights_df = pd.DataFrame([WMH_lobe_subjs, lobe_WMH_load, WMH_lobe_condition_series]).T
# WMH_lobe_weights_df.columns = ['Subject', 'wmBurden_array', 'Condition']

In [479]:
# WMH_lobe_weights_df.to_pickle('/home/riccardo/ADNI_Hopf/Results/WMH_heterogeneous_all.pkl')

In [481]:
# open file for writing, "w" 
f = open("/home/riccardo/ADNI_Hopf/Results/normalized_WMH_lobewise_all.pkl","wb")

# write json object to file
pickle.dump(lobe_WMH_normalized_dict, f)

# close file
f.close()

In [ ]:

# for subj in subjs[:1]:

#     WMH_frontal_aal = np.ones(len(aal_frontal_indices))
#     for n, WMH in np.ndenumerate(WMH_frontal_aal):
#         WMH_frontal_aal[n] = WMH_frontal_aal[n] * subjects_WMH[subj]['frontal']

#     WMH_parietal_aal = np.ones(len(aal_parietal_indices))
#     for n, WMH in np.ndenumerate(WMH_parietal_aal):
#         WMH_parietal_aal[n] = WMH_parietal_aal[n] * subjects_WMH[subj]['parietal']

#     WMH_temporal_aal = np.ones(len(aal_temporal_indices))
#     for n, WMH in np.ndenumerate(WMH_temporal_aal):
#         WMH_temporal_aal[n] = WMH_temporal_aal[n] * subjects_WMH[subj]['temporal']

#     WMH_occipital_aal = np.ones(len(aal_occipital_indices))
#     for n, WMH in np.ndenumerate(WMH_occipital_aal):
#         WMH_occipital_aal[n] = WMH_occipital_aal[n] * subjects_WMH[subj]['occipital']

#     WMH_insula_cingulate_aal = np.ones(len(aal_insula_cingulate_indices))
#     for n, WMH in np.ndenumerate(WMH_insula_cingulate_aal):
#         WMH_insula_cingulate_aal[n] = WMH_insula_cingulate_aal[n] * subjects_WMH[subj]['insula-cingulate']

# allWMH = np.hstack([WMH_frontal_aal, WMH_insula_cingulate_aal, WMH_temporal_aal, WMH_occipital_aal, WMH_parietal_aal])


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True])